In [ ]:
import os
import gradio as gr
import pandas as pd
from llama_index.llms.groq import Groq

# API KEY do GROQ
key = "YOUR KEY"

# Configuração do modelo
llm = Groq(model="llama3-70b-8192", api_key=key)

# Função para mostrar o preview do CSV
def preview_csv(csv_file):
    try:
        df = pd.read_csv(csv_file.name)
        preview = df.head()  # pega as 5 primeiras linhas
        return preview, df  # retornamos o df completo para usar na próxima etapa
    except Exception as e:
        return pd.DataFrame(), None  # retorna DataFrame vazio em caso de erro

# Função para analisar os dados com base na pergunta
def analisar_pergunta(df, pergunta):
    if df is None:
        return "Nenhum CSV carregado!"
    
    csv_text = df.to_csv(index=False)
    prompt = f"Você é um assistente de análise de dados. Baseando-se nos seguintes dados:\n\n{csv_text}\n\nResponda à pergunta: {pergunta}"

    try:
        resposta = llm.complete(prompt)
        return resposta
    except Exception as e:
        return f"Erro ao processar a pergunta: {e}"

# Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Analisador de Dados CSV com LlamaIndex + GROQ")
    
    with gr.Row():
        csv_input = gr.File(label="Escolha seu arquivo CSV")
        preview_output = gr.Dataframe(headers=None, label="Preview do CSV (5 primeiras linhas)")

    df_state = gr.State()  # Armazena o DataFrame completo

    # Atualiza o preview quando o CSV é carregado
    csv_input.upload(fn=preview_csv, inputs=csv_input, outputs=[preview_output, df_state])

    gr.Markdown("### Faça sua pergunta sobre os dados:")
    pergunta_input = gr.Textbox(label="Pergunta")
    resposta_output = gr.Textbox(label="Resposta")

    # Gera a resposta quando o usuário clica no botão
    btn = gr.Button("Perguntar")
    btn.click(fn=analisar_pergunta, inputs=[df_state, pergunta_input], outputs=resposta_output)

demo.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
